In [1]:
import numpy as np, pandas as pd
import os
from crossValidate import getXVFolds

In [79]:
z = np.random.randint(5, size=[6,3])
s = np.random.randint(3, size=6)
print(z)
print( s )

s.dot(z)

[[1 4 3]
 [3 4 0]
 [4 1 4]
 [4 3 0]
 [4 4 0]
 [0 1 4]]
[1 0 0 2 2 0]


array([17, 18,  3])

In [72]:
def concateIntercept(x):
    return np.hstack( [np.ones((x.shape[0],1), x.dtype), x] )

################################################################################

def softMax(x, wts):
    pr = np.exp(x @ wts)
    return pr / np.sum(pr,axis=1)[:,None]

def crossEntNK(x, wts, y):
    yhat = softMax(x, wts)
    err = np.sum(y*np.log(yhat), axis=1).mean()
    return err, yhat

def updateWeightNK(eta, yhat, y, x, wt):
    d = ((y - yhat).T @ x).T / len(y)
    return wt + (eta*d)

def sigmoid(x, wt):
    return 1.0 / ( 1 + np.exp(-x @ wt) )

def fitLogisticReg(x, y, eta, eps=1e-7, trace=False):
    def crossEntropy(x, wt, y):
        yhat = sigmoid(x, wt)
        err = -np.mean(y*np.log(yhat) + (1-y)*np.log(1-yhat))
        return err, yhat
    
    def updateWeight(eta, yhat, y, x, wt):
        d = (y - yhat) @ x / len(y)
        return wt + (eta*d)
    
    x = concateIntercept(x)
    wt = np.random.rand(x.shape[1])/50 - 0.01 # initialize weights
    lastErr = 0
    err,yhat = crossEntropy(x, wt, y)

    n = 0
    while (abs(err-lastErr) > eps) and n < 1e6:
        if n % 1000 == 0 and trace:
            print('Iter #%u, error: %f'%(n,err))
        wt = updateWeight(eta, yhat, y, x, wt)
        lastErr = err
        err,yhat = crossEntropy(x, wt, y)
        if err > lastErr:
            eta /= 10
        n += 1
    
    print('Final iteration #%u, error: %f' % (n-1,err) )
    return wt

IndentationError: expected an indented block (<ipython-input-72-2983fc10bf15>, line 12)

In [38]:
def discretizeMean(inDF, useMed=False):
    outDF = pd.DataFrame()
    if useMed:
        thresh = inDF.median()
    else:
        thresh = inDF.mean()
    for v in list(inDF): # loop over all columns
        outDF[v] = (inDF[v] > thresh[v]) * 1
    return outDF

# generate one-hot coding for issues with lots of missing votes
def oneHot(data, colNames):
    outDF = pd.DataFrame()
    for col in colNames:
        x = data[col]
        for val in x.unique():
            suff = 'q' if val=='?' else str(val)
            outDF[col+'_'+suff] = (x==val)*1
    return outDF


In [19]:
soyData = os.path.join('./data', 'soybean-small.data')
soyNames = ['c%02d'%(n+1) for n in range(35)] + ['class']
raw = pd.read_csv(soyData, names=soyNames)
feats = np.array(soyNames)[raw.nunique()!=1] # remove feats with only 1 value
raw = raw[feats]

In [46]:
dataMat = oneHot(raw, raw.columns[1:-1])
dataMat['class'] = raw['class']

In [51]:
dataMat.nunique()

c02_0    2
c02_1    2
c03_2    2
c03_0    2
c03_1    2
c04_1    2
c04_2    2
c04_0    2
c05_1    2
c05_0    2
c06_1    2
c06_3    2
c06_2    2
c06_0    2
c07_0    2
c07_1    2
c07_2    2
c07_3    2
c08_1    2
c08_2    2
c09_0    2
c09_1    2
c10_2    2
c10_1    2
c10_0    2
c12_1    2
c12_0    2
c20_0    2
c20_1    2
c21_3    2
c21_0    2
c21_1    2
c21_2    2
c22_1    2
c22_0    2
c22_3    2
c22_2    2
c23_1    2
c23_0    2
c24_1    2
c24_0    2
c25_0    2
c25_1    2
c26_0    2
c26_2    2
c27_0    2
c27_1    2
c28_0    2
c28_3    2
c35_0    2
c35_1    2
class    4
dtype: int64

In [ ]:
vote84Data = os.path.join('./data', 'house-votes-84.data')
vote84Names = ['party', 'infant', 'water', 'budget', 'doctorfee','salvador',
              'religion', 'satellite', 'contras', 'missile', 'immigration',
              'synfuels', 'education', 'superfund', 'crime', 'exports',
              'ZAF']
raw = pd.read_csv(vote84Data , names=vote84Names ) # read in vote file

oneHotCols = pd.concat([oneHot(raw,'water'), oneHot(raw,'education'), 
                        oneHot(raw,'ZAF')], axis=1)

# remove variables with completed one-hot coding from list of variables
yesVars = np.setdiff1d(vote84Names[1:],['water','education','ZAF'])
yesVote = raw.loc[:,yesVars] == 'y' # boolean for vote='yes' for rest of vars
yesVote.columns = [s+'_y' for s in yesVote.columns]
repub = raw.loc[:,['party']] == 'republican' # boolean for republicans
voteData = pd.concat([yesVote,oneHotCols], axis=1) # concat two dataframes
voteMat = voteData.values * 1 # give matrixs of 0 & 1 for calculation
repubVec = repub.values.ravel() * 1 # vector of 0 & 1 for calculation

In [ ]:
folds = getXVFolds(voteMat, repubVec, categorical=True)
trainIdx = folds[0]
testIdx = np.hstack(folds[1:])
trainData,trainLabel = voteMat[trainIdx],repubVec[trainIdx]
testData,testLabel = voteMat[testIdx],repubVec[testIdx]

In [ ]:
wts = fitLogisticReg(trainData, trainLabel, 0.1, trace=True)

In [ ]:
np.mean((sigmoid(concateIntercept(testData), wts)>0.5)*1==testLabel)